In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('data/melon_song.csv')
df.head(1)

,songId,title,artist,genre,album,lyricist,composer,date,img,comment,like,lyric,plylstSeq
0,418168,희재,성시경,발라드 국내영화,국화꽃 향기 OST,양재선,MGR,20030201.0,https://cdnimg.melon.co.kr/cm/album/images/000...,332,138267,햇살은 우릴 위해 내리고 \n바람도 서롤 감싸게 했죠 \n우리 웃음속에 계절은 오고...,445029956 411111859


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6247 entries, 0 to 6246
Data columns (total 13 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   songId     6247 non-null   int64  
 1   title      6247 non-null   object 
 2   artist     6247 non-null   object 
 3   genre      6245 non-null   object 
 4   album      6247 non-null   object 
 5   lyricist   5849 non-null   object 
 6   composer   5845 non-null   object 
 7   date       6202 non-null   float64
 8   img        6247 non-null   object 
 9   comment    6247 non-null   int64  
 10  like       6247 non-null   int64  
 11  lyric      6247 non-null   object 
 12  plylstSeq  6126 non-null   object 
dtypes: float64(1), int64(3), object(9)
memory usage: 634.6+ KB


In [4]:
#  1. 컨텐츠기반 추천
#    Input(title, artist) : contains 로 찾기 => 결과 songId 한 개만 반환하도록 한다.
#    songId로 추천시스템에서 비슷한 노래 5개 찾아서 songId로 반환
#    반환된 songId 들의 정보 출력

# 가사에서 \r -> replace하기

In [5]:
df['songId'] = df.songId.astype(str)

In [6]:
df.fillna('', inplace=True)

In [7]:
# df[df.title.str.contains('희재') & df.artist.str.contains('성시경')]

In [8]:
# title, artist = map(input().split())

# df[df.title.str.contains('title') & df.artist.str.contains('artist')]

In [9]:
def songid_find(title, artist):
    ss = df[df.title.str.contains(title) & df.artist.str.contains(artist)]
    return ss.songId

In [10]:
songid_find('희재','성시경')

0     418168
55    418200
Name: songId, dtype: object

In [11]:
df['total'] = df.lyric + (' ' + df.artist) * 2 + (' ' + df.composer) * 2 + (' ' + df.lyricist) * 2 + (' ' + df.genre) * 2 + (' ' + df.title)
df.total[0]

'햇살은 우릴 위해 내리고 \n바람도 서롤 감싸게 했죠 \n우리 웃음속에 계절은 오고 \n또 갔죠 \n바람에 흔들리는 머릿결 \n내게 불어오는 그대 향기 \n예쁜 두눈도 웃음 소리도 \n모두가 내 것이었죠\n이런 사랑 이런 행복 쉽다 \n했었죠 이런 웃음 \n이런 축복 내게 쉽게 올리 없죠 \n눈물 조차 울음조차 닦지 \n못한 나 \n정말로 울면 내가 그댈 보내 \n준 것 같아서 \n그대 떠나가는 그 순간도 \n나를 걱정했었나요 \n무엇도 해줄 수 없는 내 맘\n앞에서 \n그대 나를 떠나 간다해도 \n난 그댈 보낸적 없죠 \n여전히 그댄 나를 살게 하는 \n이율테니 \n이런 사랑 이런 행복 쉽다 했었죠 \n이런 웃음 이런 축복 내게 \n쉽게 올리 없죠 \n눈물 조차 울음 조차 닦지 못한 나 \n정말로 울면 내가 그댈 보내 \n준 것 같아서 \n그대 떠나가는 그 순간도 \n나를 걱정했었나요 \n무엇도 해줄 수 없는 내 맘앞에서 \n그대 나를 떠나 간다해도 난 \n그댈 보낸적 없죠 \n기다림으로 다시 시작일테니 \n얼마나 사랑했는지 얼마나 \n또 울었는지 \n그대여 한순간 조차 잊지 말아요 \n거기 떠나간 그곳에서 날\n기억하며 기다려요\n하루씩 그대에게 다가가는 \n나 일테니 성시경 성시경 MGR MGR 양재선 양재선 발라드 국내영화 발라드 국내영화 희재'

In [12]:
df.lyric[0]

'햇살은 우릴 위해 내리고 \n바람도 서롤 감싸게 했죠 \n우리 웃음속에 계절은 오고 \n또 갔죠 \n바람에 흔들리는 머릿결 \n내게 불어오는 그대 향기 \n예쁜 두눈도 웃음 소리도 \n모두가 내 것이었죠\n이런 사랑 이런 행복 쉽다 \n했었죠 이런 웃음 \n이런 축복 내게 쉽게 올리 없죠 \n눈물 조차 울음조차 닦지 \n못한 나 \n정말로 울면 내가 그댈 보내 \n준 것 같아서 \n그대 떠나가는 그 순간도 \n나를 걱정했었나요 \n무엇도 해줄 수 없는 내 맘\n앞에서 \n그대 나를 떠나 간다해도 \n난 그댈 보낸적 없죠 \n여전히 그댄 나를 살게 하는 \n이율테니 \n이런 사랑 이런 행복 쉽다 했었죠 \n이런 웃음 이런 축복 내게 \n쉽게 올리 없죠 \n눈물 조차 울음 조차 닦지 못한 나 \n정말로 울면 내가 그댈 보내 \n준 것 같아서 \n그대 떠나가는 그 순간도 \n나를 걱정했었나요 \n무엇도 해줄 수 없는 내 맘앞에서 \n그대 나를 떠나 간다해도 난 \n그댈 보낸적 없죠 \n기다림으로 다시 시작일테니 \n얼마나 사랑했는지 얼마나 \n또 울었는지 \n그대여 한순간 조차 잊지 말아요 \n거기 떠나간 그곳에서 날\n기억하며 기다려요\n하루씩 그대에게 다가가는 \n나 일테니'

In [13]:
df.set_index('songId', inplace=True)
df.reset_index(inplace=True)
df.total[0]

'햇살은 우릴 위해 내리고 \n바람도 서롤 감싸게 했죠 \n우리 웃음속에 계절은 오고 \n또 갔죠 \n바람에 흔들리는 머릿결 \n내게 불어오는 그대 향기 \n예쁜 두눈도 웃음 소리도 \n모두가 내 것이었죠\n이런 사랑 이런 행복 쉽다 \n했었죠 이런 웃음 \n이런 축복 내게 쉽게 올리 없죠 \n눈물 조차 울음조차 닦지 \n못한 나 \n정말로 울면 내가 그댈 보내 \n준 것 같아서 \n그대 떠나가는 그 순간도 \n나를 걱정했었나요 \n무엇도 해줄 수 없는 내 맘\n앞에서 \n그대 나를 떠나 간다해도 \n난 그댈 보낸적 없죠 \n여전히 그댄 나를 살게 하는 \n이율테니 \n이런 사랑 이런 행복 쉽다 했었죠 \n이런 웃음 이런 축복 내게 \n쉽게 올리 없죠 \n눈물 조차 울음 조차 닦지 못한 나 \n정말로 울면 내가 그댈 보내 \n준 것 같아서 \n그대 떠나가는 그 순간도 \n나를 걱정했었나요 \n무엇도 해줄 수 없는 내 맘앞에서 \n그대 나를 떠나 간다해도 난 \n그댈 보낸적 없죠 \n기다림으로 다시 시작일테니 \n얼마나 사랑했는지 얼마나 \n또 울었는지 \n그대여 한순간 조차 잊지 말아요 \n거기 떠나간 그곳에서 날\n기억하며 기다려요\n하루씩 그대에게 다가가는 \n나 일테니 성시경 성시경 MGR MGR 양재선 양재선 발라드 국내영화 발라드 국내영화 희재'

In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer
tvect = TfidfVectorizer()
total_tv = tvect.fit_transform(df.total)
total_tv.shape

(6247, 95076)

In [15]:
indices = pd.Series(df.index, index=df.songId)
indices

songId
418168         0
35609035       1
1046278        2
35609034       3
35728845       4
            ... 
33013736    6242
3080890     6243
2981597     6244
34183979    6245
31191637    6246
Length: 6247, dtype: int64

In [16]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_sim = cosine_similarity(total_tv)

In [17]:
def get_recommendation(songId, cos_sim):
    index = indices[songId]
    sim_scores = pd.Series(cos_sim[index])
    song_indices = sim_scores.sort_values(ascending=False).head(6).tail(5).index
    return df.songId.iloc[song_indices]

In [18]:
a = get_recommendation('35609035', cosine_sim)
a.values

array(['35609034', '35307770', '4381724', '1925230', '32323332'],
      dtype=object)

In [19]:
a.values[0]

'35609034'

In [20]:
df[df['songId'].isin(a)]

,songId,title,artist,genre,album,lyricist,composer,date,img,comment,like,lyric,plylstSeq,total
3,35609034,슬픈등,김호중,발라드 국내영화,인생은 뷰티풀: 비타돌체,김호중,김호중,20220907.0,https://cdnimg.melon.co.kr/cm2/album/images/11...,145,15389,누군가를 다시 만난다면\n여전히 나는 너를\n누군가를 다시 사랑하면\n영원히 나는 ...,457536944 515278519,누군가를 다시 만난다면\n여전히 나는 너를\n누군가를 다시 사랑하면\n영원히 나는 ...
1593,1925230,저꽃속에 찬란한 빛이,박경희,성인가요/트로트,김기웅 걸작선,전우,김기웅,19920701.0,https://cdnimg.melon.co.kr/cm/album/images/003...,1,25,어둠에 묻혀 흘러간 그 세월의 눈물은\n사랑을 잃어 흩어진 옛 추억의 그림자\n잊을...,460454143,어둠에 묻혀 흘러간 그 세월의 눈물은\n사랑을 잃어 흩어진 옛 추억의 그림자\n잊을...
3201,4381724,그대가 분다,엠씨더맥스 (M.C the MAX),발라드,UNVEILING,최성일,민연재,20140102.0,https://cdnimg.melon.co.kr/cm/album/images/022...,110,107402,옅은 바람 냄새에 \n두 눈을 감아본다\n낡은 가슴 여전한 떨림 \n그대가 그대가 ...,403014132 445029956,옅은 바람 냄새에 \n두 눈을 감아본다\n낡은 가슴 여전한 떨림 \n그대가 그대가 ...
3673,35307770,빛이 나는 사람,김호중,발라드,빛이 나는 사람,김호중,김호중,20220618.0,https://cdnimg.melon.co.kr/cm2/album/images/10...,545,20680,내가 긴밤을 헤매고 있을때\n때론 슬픔에 잠겨있을때\n거친 세상에 지독한 외로움속에...,482887327 476517031,내가 긴밤을 헤매고 있을때\n때론 슬픔에 잠겨있을때\n거친 세상에 지독한 외로움속에...
5691,32323332,태클을 걸지마,김호중,성인가요/트로트,내일은 미스터트롯 예선곡 베스트,진성,진성철,20200117.0,https://cdnimg.melon.co.kr/cm2/album/images/10...,1265,26549,어떻게 살았냐고 묻지를 마라\n이리저리 살았을 거란\n착각도 마라\n그래 한때 삶에...,475427896 478284381,어떻게 살았냐고 묻지를 마라\n이리저리 살았을 거란\n착각도 마라\n그래 한때 삶에...


In [24]:
filtered_df = df[df['songId'].isin(a)]
# filtered_df.to_dict('recodes')
filtered_df.to_dict()

{'songId': {3: '35609034',
  1593: '1925230',
  3201: '4381724',
  3673: '35307770',
  5691: '32323332'},
 'title': {3: '슬픈등',
  1593: '저꽃속에 찬란한 빛이',
  3201: '그대가 분다',
  3673: '빛이 나는 사람',
  5691: '태클을 걸지마'},
 'artist': {3: '김호중',
  1593: '박경희',
  3201: '엠씨더맥스 (M.C the MAX)',
  3673: '김호중',
  5691: '김호중'},
 'genre': {3: '발라드 국내영화',
  1593: '성인가요/트로트',
  3201: '발라드',
  3673: '발라드',
  5691: '성인가요/트로트'},
 'album': {3: '인생은 뷰티풀: 비타돌체',
  1593: '김기웅 걸작선',
  3201: 'UNVEILING',
  3673: '빛이 나는 사람',
  5691: '내일은 미스터트롯 예선곡 베스트'},
 'lyricist': {3: '김호중', 1593: '전우', 3201: '최성일', 3673: '김호중', 5691: '진성'},
 'composer': {3: '김호중', 1593: '김기웅', 3201: '민연재', 3673: '김호중', 5691: '진성철'},
 'date': {3: 20220907.0,
  1593: 19920701.0,
  3201: 20140102.0,
  3673: 20220618.0,
  5691: 20200117.0},
 'img': {3: 'https://cdnimg.melon.co.kr/cm2/album/images/110/48/906/11048906_20220906180454_500.jpg?d530d42b8ca05e5fe8c89007570a3d26/melon/resize/282/quality/80/optimize',
  1593: 'https://cdnimg.melon.co.kr/cm/album/